In [84]:
from BRScraper import nba
import pandas as pd 
import numpy as np

In [87]:
desired_seasons = ['2023-24', '2022-23', '2021-22'] # seasons to gather salaries for, will always include 1 season before the oldest listed
salaries = nba.get_current_salaries() # get salaries for current season only
salaries = salaries.drop_duplicates('Player').reset_index(drop=True)
salaries = salaries.drop(['Tm', 'Guaranteed'], axis = 1)

salaries

,Player,2023-24,2024-25,2025-26,2026-27,2027-28,2028-29
0,Stephen Curry,"$51,915,615","$55,761,216","$59,606,817",NaN,NaN,NaN
1,Kevin Durant,"$47,649,433","$51,179,021","$54,708,609",NaN,NaN,NaN
2,Nikola Jokić,"$47,607,350","$51,415,938","$55,224,526","$59,033,114","$62,841,702",NaN
3,Joel Embiid,"$47,607,350","$51,415,938","$55,224,526","$59,033,114",NaN,NaN
4,LeBron James,"$47,607,350","$51,415,938",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
464,Nerlens Noel,"$300,000",NaN,NaN,NaN,NaN,NaN
465,Didi Louzada,"$268,032","$268,032","$268,032","$268,032","$268,032","$268,032"
466,Neemias Queta,"$250,000",NaN,NaN,NaN,NaN,NaN
467,Danny Green,"$200,000",NaN,NaN,NaN,NaN,NaN


In [88]:
for season in desired_seasons:
    url = f'https://hoopshype.com/salaries/players/{int(season[:4]) - 1}-20{int(season[5:7]) - 1}'

    previous_season = pd.read_html(url)[0]
    previous_season = previous_season.drop(['Unnamed: 0', f'{int(season[:4]) - 1}/{int(season[5:7]) - 1}(*)'], axis = 1) # drop unneeded columns
    previous_season = previous_season.rename(columns = {f'{int(season[:4]) - 1}/{int(season[5:7]) - 1}' : f'{int(season[:4]) - 1}-{int(season[5:7]) - 1}'})
    # rename columns to match format in 'salaries'

    players = salaries[salaries['Player'].isin(previous_season['Player']) == False] # get only the players that had played in the previous season

    previous_season['Player'] = previous_season['Player'].str.replace('.','') # remove periods from names
    previous_season['Player'] = previous_season['Player'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    previous_season = previous_season.drop_duplicates('Player', keep='first').reset_index(drop=True)

    salaries = previous_season.merge(salaries, on='Player', how='left', validate='1:1') # merge different years together
    salaries = pd.concat([salaries, players], ignore_index=True) # merge removing players that did not play in the previous season

for col in salaries.columns:
    if col not in ['Player']:
        salaries[col] = (salaries[col].str.replace('$','').str.replace(',','')).astype(float) # remove dollar signs
        salaries[col] = salaries[col].fillna(-1000000) # set all cells without value to negative one million
        salaries[col] = salaries[col] / 1000000 

salaries.to_csv('salaries.csv', sep = ',', decimal = '.', index = False)
salaries.head()

,Player,2020-21,2021-22,2022-23,2023-24,2024-25,2025-26,2026-27,2027-28,2028-29
0,Stephen Curry,43.006362,45.780966,48.070014,51.915615,55.761216,59.606817,-1.0,-1.0,-1.0
1,Chris Paul,41.358814,30.800000,28.400000,30.800000,30.000000,-1.000000,-1.0,-1.0,-1.0
2,Russell Westbrook,41.358814,44.211146,47.080179,3.835738,4.027525,-1.000000,-1.0,-1.0,-1.0
3,James Harden,41.254920,44.310840,33.000000,35.640000,-1.000000,-1.000000,-1.0,-1.0,-1.0
4,John Wall,41.254920,44.310840,47.345760,6.802950,-1.000000,-1.000000,-1.0,-1.0,-1.0
